# Testing Folium for Creating Interactive Maps
After: https://github.com/python-visualization/folium

Patty Frontiera - pattyf@berkeley.edu 07/21/2015

In [1]:
import folium
from IPython.display import HTML

In [2]:
def inline_map(m, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: {1}px; '
                 'border: none"></iframe>'.format(srcdoc, height))
    return embed

In [3]:
# simple marker map

#width, height = 800, 400
#map1 = folium.Map(location=[40, -97], zoom_start=3,
#                    tiles='OpenStreetMap', width=width, height=height)
map1 = folium.Map(location=[40, -97], zoom_start=2,
                    tiles='OpenStreetMap')
map1.simple_marker(location=[46.8354, -121.7325], popup='Camp Muir')
inline_map(map1)

In [6]:

# simple line map

#width, height = 800, 400
#map1 = folium.Map(location=[40, -97], zoom_start=3,
#                    tiles='OpenStreetMap', width=width, height=height)
map1 = folium.Map(location=[40, -97], zoom_start=2,
                    tiles='OpenStreetMap')
map1.line(locations=[(45.5, -122.3), (42.3, -71.0)],line_color='red', line_opacity=1.0)
inline_map(map1)

In [7]:
# simple line map
sfo = (37.619,-122.376) # San Francisco International Airport
lhr = (51.471, -0.456)  # London Heathrow Airport

map1 = folium.Map(location=[40, -97], zoom_start=2,
                    tiles='OpenStreetMap')

map1.line(locations=[sfo, lhr],line_color='purple', line_opacity=1.0, line_weight=2)
inline_map(map1)

In [13]:
# Simple two layer reference map
geo_path = r'data/antarctic_ice_edge.json'
#topo_path = r'data/antarctic_ice_shelf_topo.json'

#geo_path = r'http://eric.clst.org/wupl/Stuff/gz_2010_us_outline_500k.json'

ice_map = folium.Map(location=[-59.1759, -11.6016],
                   tiles='Mapbox Bright', zoom_start=2)
ice_map.geo_json(geo_path=geo_path, line_color='red')
#ice_map.geo_json(geo_path=topo_path, topojson='objects.antarctic_ice_shelf')
inline_map(ice_map)

1.4142135623730951

In [8]:
state_geo = r'data/us-states.json'
state_map = folium.Map(location=[48, -102], zoom_start=3)
state_map.geo_json(geo_path=state_geo)
inline_map(state_map)

In [9]:
#zip_geo = r'data/ca_zips_topo.json'
#zip_map = folium.Map(location=[48, -102], zoom_start=3)
#zip_map.geo_json(geo_path=zip_geo, topojson='objects.ca_zipsonly')
#inline_map(zip_map)

zip_geo = r'data/ca_zipsbycode_topo.json'
zip_map = folium.Map(location=[48, -102], zoom_start=3)
zip_map.geo_json(geo_path=zip_geo, topojson='objects.ca_zip_polys')
inline_map(zip_map)
#ca_zip_polys.geojson


### Choropleth maps
using pandas and class interval code

In [10]:
import pandas as pd
from class_intervals_only import * 
## class intervals can be one of equal, quantile, std_dev, jenks, jenks_sample, and pretty


In [11]:

state_unemployment = r'data/US_Unemployment_Oct2012.csv'

state_data = pd.read_csv(state_unemployment)

#Let Folium determine the scale
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data,
             columns=['State', 'Unemployment'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Unemployment Rate (%)')
inline_map(map2)

In [12]:
map = folium.Map(location=[48, -102], zoom_start=3)

# manually set the scale
map.geo_json(geo_path=state_geo, data=state_data,
             columns=['State', 'Unemployment'],
             #threshold_scale=[5, 6, 7, 8, 9, 10],
             threshold_scale=[4, 5, 6, 7, 8, 10],
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,
             legend_name='Unemployment Rate (%)',
             reset=True)
inline_map(map)

In [13]:
# Use class_intervals.py to define the scales

map = folium.Map(location=[48, -102], zoom_start=3)

myvals = state_data['Unemployment'].values
print(type(myvals))
colcuts = [ round(elem, 2) for elem in quantile(myvals) ]
#colcuts= quantile(myvals)

map.geo_json(geo_path=state_geo, data=state_data,
    columns=['State', 'Unemployment'],
    threshold_scale=colcuts,
    key_on='feature.id',
    fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,
    legend_name='Unemployment Rate (%)',
    reset=True)
  
inline_map(map)
    

<type 'numpy.ndarray'>


In [14]:
# use class_intervals.py to define the scales
 
def make_choro_map(cut_type=quantile):
    myvals = state_data['Unemployment'].values
    colcuts = [ round(elem, 2) for elem in cut_type(myvals) ]

    map = folium.Map(location=[48, -102], zoom_start=3)
    map.geo_json(geo_path=state_geo, data=state_data,
        columns=['State', 'Unemployment'],
        threshold_scale=colcuts,
        key_on='feature.id',
        fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,
        legend_name='Unemployment Rate (%)',
        reset=True)
    return map


In [15]:
mymap = make_choro_map()
inline_map(mymap)


In [16]:

inline_map(make_choro_map(equal))


In [17]:
inline_map(make_choro_map(pretty))

### Next examples
Uses county data which makes for more interesting choropleth maps

In [18]:
 
county_data = r'us_county_data.csv'
county_geo = r'us_counties_20m_topo.json'

#Read into Dataframe, cast to string for consistency
df = pd.read_csv(county_data, na_values=[' '])
df['FIPS_Code'] = df['FIPS_Code'].astype(str)

def set_id(fips):
    '''Modify FIPS code to match GeoJSON property'''
    if fips == '0':
        return None
    elif len(fips) <= 4:
        return ''.join(['0500000US0', fips])
    else:
        return ''.join(['0500000US', fips])

#Apply set_id, drop NaN
df['GEO_ID'] = df['FIPS_Code'].apply(set_id)
df = df.dropna()
myvals = df['Unemployment_rate_2011'].values
colcuts= equal(myvals)  
   
map_2 = folium.Map(location=[40, -99], zoom_start=4,tiles='OpenStreetMap')
map_2.geo_json(geo_path=county_geo, data_out='data2.json', data=df,
               columns=['GEO_ID', 'Unemployment_rate_2011'],
               key_on='feature.id',
               #threshold_scale=[0, 5, 7, 9, 11, 13],
               threshold_scale= colcuts,
               fill_color='YlGnBu', line_opacity=0.1,
               legend_name='Unemployment Rate 2011 (%)',
               topojson='objects.us_counties_20m',
              reset=True)
inline_map(map_2)

In [19]:
# try different class interval scheme
colcuts= quantile(myvals)  
   
map_2 = folium.Map(location=[40, -99], zoom_start=4,tiles='OpenStreetMap')
map_2.geo_json(geo_path=county_geo, data_out='data2.json', data=df,
               columns=['GEO_ID', 'Unemployment_rate_2011'],
               key_on='feature.id',
               #threshold_scale=[0, 5, 7, 9, 11, 13],
               threshold_scale= colcuts,
               fill_color='YlGnBu', line_opacity=0.1,
               legend_name='Unemployment Rate 2011 (%)',
               topojson='objects.us_counties_20m',
              reset=True)
inline_map(map_2)

In [22]:
# try different class interval scheme
colcuts= jenks(myvals)  
   
map_2 = folium.Map(location=[40, -99], zoom_start=4,tiles='OpenStreetMap')
map_2.geo_json(geo_path=county_geo, data_out='data2.json', data=df,
               columns=['GEO_ID', 'Unemployment_rate_2011'],
               key_on='feature.id',
               #threshold_scale=[0, 5, 7, 9, 11, 13],
               threshold_scale= colcuts,
               fill_color='YlGnBu', fill_opacity=1, line_opacity=0.8,line_color='white',
               legend_name='Unemployment Rate 2011 (%)',
               topojson='objects.us_counties_20m',
              reset=True)
inline_map(map_2)